In [166]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import csv

import torch
import captum
from captum.attr import IntegratedGradients
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
import re
import json



# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Topology Defination

In [167]:
NoOfCUs = 4
NoOfDUs = 4

# Creating Topology
topology = {}

# Form the graph where srscu0 connects to srsdu0, srscu1 to srsdu1, and so on
for i in range(min(NoOfCUs, NoOfDUs)):  # Prevent index errors
    topology[f"srscu{i}"] = [f"srsdu{i}"]

UEsOfDUs = {}

for i in range(NoOfDUs):
    UEsOfDUs[f"srsdu{i}"] = []


# Display the graph
print(f'Topology is as follows: \n{topology}', end="\n\n")

# Load dataset
dataset = pd.read_csv('small_sample.csv')
# dataset = dataset[:int(0.01*len(dataset))]

dataset.index = dataset['Timestamp']
dataset = dataset.drop(columns=['Timestamp'])

# for column in dataset.columns:
#     if 'PCI' in column:
#         print(column)

# Dictionary to store for each PCI:
# a list of RNTIs and a list of metric types
pci_info_map = defaultdict(lambda: {'rntis': set(), 'metrics': set()})


# Process each column
for column in dataset.columns:
    match = re.match(r'PCI-(\d+)_RNTI-(\d+)_([a-zA-Z0-9_]+)', column)
    if match:
        pci = match.group(1)
        rnti = match.group(2)
        metric = match.group(3)
        pci_info_map[pci]['rntis'].add(rnti)
        pci_info_map[pci]['metrics'].add(metric)

# # Print results
# for pci, info in pci_info_map.items():
#     print(f"PCI-{pci}:")
#     print(f"  RNTIs: {sorted(info['rntis'])}")
#     print(f"  Metrics: {sorted(info['metrics'])}\n\t   Length: {len(info['metrics'])}")


application_features = pci_info_map['1'][ 'metrics']
print(f"Application Features: {application_features}")


Topology is as follows: 
{'srscu0': ['srsdu0'], 'srscu1': ['srsdu1'], 'srscu2': ['srsdu2'], 'srscu3': ['srsdu3']}

Application Features: {'dl_nof_ok', 'ri', 'ul_nof_ok', 'dl_nof_nok', 'srs_ta_ns', 'ul_mcs', 'pucch_ta_ns', 'pucch_snr_db', 'ul_brate', 'dl_mcs', 'dl_brate', 'pusch_snr_db', 'cqi', 'bsr', 'pusch_ta_ns', 'ul_nof_nok', 'dl_bs', 'ta_ns'}


In [168]:
node_features = [ col for col in dataset.columns.tolist() if 'node' in col ]
features_container_wise = {}
application_features_for_each_DU = {}

for CU, connected_DUs in topology.items():
    features_container_wise[CU] = []
    for DU in connected_DUs:
        features_container_wise[DU] = []
    # Add node features for CU
    features_container_wise[CU].extend([col for col in dataset.columns if CU in col])
    # Add DU features for each connected DU
    for DU in connected_DUs:
        # Extract features for the DU
        DU_features = [col for col in dataset.columns if DU in col]
        features_container_wise[DU].extend(DU_features)



for CU, connected_DUs in topology.items():
    for DU in connected_DUs:
        du_index = str(int(DU.replace('srsdu', '')) + 1)  # PCI is assumed 0-based, matching DU index
        rntis = pci_info_map[du_index]['rntis']
        metrics = pci_info_map[du_index]['metrics']
        for metric in metrics:
            for rnti in rntis:
                feature = f'PCI-{du_index}_RNTI-{rnti}_{metric}'
                if feature in dataset.columns:
                    if DU not in application_features_for_each_DU:
                        application_features_for_each_DU[DU] = []
                    application_features_for_each_DU[DU].append(feature)

# # Print the features for each DU
# print("Application features for each DU:")
# for DU, features in application_features_for_each_DU.items():
#     print(f"{DU}: {features}\n\t{len(features)} features")


# print("application_features_for_each_DU")
# for DU, features in application_features_for_each_DU.items():
#     print(f"{DU}: {features}\n\t{len(features)} features")


# print("Features container wise:")
# for container, features in features_container_wise.items():
#     print(f"{container}: {features}\n\t{len(features)} features")

In [169]:
features_by_CU = {}

for CU, connected_DUs in topology.items():
    for DU in connected_DUs:
        features = []
        features = features_container_wise[CU] + features_container_wise[DU] + node_features
        features = features + application_features_for_each_DU[DU]
        features = sorted(set(features))
        features_by_CU[(CU, DU)] = features
    if(len(features) == len(set(features))):
        pass
    else:
        print(f"Duplicate features found for {CU} and {DU}")

    # Save to JSON
    with open(f'features_{CU}_{DU}.json', 'w') as f:
        json.dump(features, f, indent=2)
    
    # # read the JSON file to verify
    # with open(f'features_{CU}_{DU}.json', 'r') as f:
    #     loaded_features = json.load(f)
    #     print(f"Loaded features for {CU} and {DU}: {loaded_features}\n\tLength: {len(loaded_features)} features")


# print(features_by_CU)